In [81]:
%matplotlib inline
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

from sklearn.model_selection import train_test_split, GridSearchCV, StratifiedKFold
from sklearn.preprocessing import StandardScaler, Imputer
from sklearn.tree import DecisionTreeClassifier
from sklearn.pipeline import make_pipeline

In [82]:
url= "https://raw.githubusercontent.com/BigDataGal/Python-for-Data-Science/master/titanic-train.csv"

In [83]:
titanic = pd.read_csv(url)

In [84]:
titanic.head()

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S


from the above we'll choose some selected column to build a dataset.<br>
'Survived' column will be the target set

In [85]:
label = titanic['Survived'].values        #target
features = titanic[['Pclass', 'Sex', 'Age', 'SibSp', 'Parch', 'Fare']]    #data set

In [86]:
features.head()

,Pclass,Sex,Age,SibSp,Parch,Fare
0,3,male,22.0,1,0,7.2500
1,1,female,38.0,1,0,71.2833
2,3,female,26.0,0,0,7.9250
3,1,female,35.0,1,0,53.1000
4,3,male,35.0,0,0,8.0500


The data now contains only useful features, but they are not in a format that the machine learning algorithms can understand. We need to encode the categorical variables
We can do that using the pandas ``get_dummies`` function:

In [87]:
featurs_dummies = pd.get_dummies(features, columns=['Pclass', 'Sex'])     #
featurs_dummies.head()

,Age,SibSp,Parch,Fare,Pclass_1,Pclass_2,Pclass_3,Sex_female,Sex_male
0,22.0,1,0,7.2500,0,0,1,0,1
1,38.0,1,0,71.2833,1,0,0,1,0
2,26.0,0,0,7.9250,0,0,1,1,0
3,35.0,1,0,53.1000,1,0,0,1,0
4,35.0,0,0,8.0500,0,0,1,0,1


In [88]:
data = featurs_dummies.values
data                            #data sets for trianing and testing purpose

array([[22.,  1.,  0., ...,  1.,  0.,  1.],
       [38.,  1.,  0., ...,  0.,  1.,  0.],
       [26.,  0.,  0., ...,  1.,  1.,  0.],
       ...,
       [nan,  1.,  2., ...,  1.,  1.,  0.],
       [26.,  0.,  0., ...,  0.,  0.,  1.],
       [32.,  0.,  0., ...,  1.,  0.,  1.]])

In [89]:
x_train, x_test, y_train, y_test = train_test_split(data, label, 
                                                    test_size = 0.25,
                                                    random_state = 42,
                                                    stratify=label)

In [107]:
pipe = make_pipeline(Imputer(),                        #preprocessing
                    StandardScaler(),                  #preprocessing
                    DecisionTreeClassifier())          #estimator

cv = StratifiedKFold(n_splits=5, random_state=42)        #cross validation
param_grid = {'decisiontreeclassifier__criterion': ['gini', 'entropy'], 'decisiontreeclassifier__max_depth': [3, 5, 7, 9, 13]}

grid = GridSearchCV(pipe, param_grid, cv=5)

In [109]:
grid.fit(x_train, y_train)          #training 

GridSearchCV(cv=5, error_score='raise',
       estimator=Pipeline(memory=None,
     steps=[('imputer', Imputer(axis=0, copy=True, missing_values='NaN', strategy='mean', verbose=0)), ('standardscaler', StandardScaler(copy=True, with_mean=True, with_std=True)), ('decisiontreeclassifier', DecisionTreeClassifier(class_weight=None, criterion='gini', max_depth=None,
            max_featu...      min_weight_fraction_leaf=0.0, presort=False, random_state=None,
            splitter='best'))]),
       fit_params=None, iid=True, n_jobs=1,
       param_grid={'decisiontreeclassifier__criterion': ['gini', 'entropy'], 'decisiontreeclassifier__max_depth': [3, 5, 7, 9, 13]},
       pre_dispatch='2*n_jobs', refit=True, return_train_score='warn',
       scoring=None, verbose=0)

In [110]:
grid.score(x_test, y_test)       #accuracy of model

0.7713004484304933

In [111]:
grid.best_params_        #best combination of decision tree parameter which results above accuracy

{'decisiontreeclassifier__criterion': 'gini',
 'decisiontreeclassifier__max_depth': 5}